In [1]:
import random
import string
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

In [2]:
# 1) AES 클래스 구현 
class AESCipher:
    # mode는 정수형태의 상수로 구성
    MODE_ECB = 1  # ECB 모드
    MODE_CBC = 2  # CBC 모드
    MODE_OFB = 3  # OFB 모드
    
    def __init__(self):
        self.blocksize = 16  # 블록 사이즈
        self.mode = AES.MODE_ECB  # 기본 모드로 ECB 설정

    def Set_Mode(self, mode):
        # 주어진 모드에 따라 암호화 모드 설정
        if mode == self.MODE_ECB:
            self.mode = AES.MODE_ECB
        elif mode == self.MODE_CBC:
            self.mode = AES.MODE_CBC
        elif mode == self.MODE_OFB:
            self.mode = AES.MODE_OFB
        else:
            # 유효하지 않은 모드일 경우 오류 발생
            raise ValueError("잘못된 AES 모드입니다.")

    def Create_IV(self):
        # 초기화 벡터 생성 : IV는 숫자, 영문자 대소문자로 구성된 16바이트 String
        return ''.join(random.choices(string.ascii_letters + string.digits, k=self.blocksize))

    def Create_Key(self, key_size=16):
        # key는 16/24/32 바이트 값만 지원
        if key_size not in [16, 24, 32]:
            raise ValueError("잘못된 Key size입니다.")
        # 키 생성
        return ''.join(random.choices(string.ascii_letters + string.digits, k=key_size))

    def pad(self, s):
        # 블록 사이즈에 맞게 패딩
        return s + (self.blocksize - len(s) % self.blocksize) * chr(self.blocksize - len(s) % self.blocksize).encode()

    def unpad(self, s):
        # 패딩 제거
        return s[:-ord(s[len(s) - 1:])]

    def Encrypt(self, plain_text, key, iv=None):
        # CBC나 OFB 모드에서 IV가 필요한 경우 검증
        if self.mode != AES.MODE_ECB and iv is None:
            raise ValueError("CBC 및 OFB 모드에는 IV가 필요합니다.")

        data = self.pad(plain_text.encode())
        
        # ECB 모드일 경우
        if self.mode == AES.MODE_ECB:
            cipher = AES.new(key.encode(), self.mode)
            encrypted = cipher.encrypt(data)
        else:  # CBC나 OFB 모드일 경우
            cipher = AES.new(key.encode(), self.mode, iv.encode())
            encrypted = cipher.encrypt(data)

        # 결과 반환 : cipher_text를 return
        return encrypted.hex()

    def Decrypt(self, cipher_text, key, iv=None):
        # CBC나 OFB 모드에서 IV가 필요한 경우 검증
        if self.mode != AES.MODE_ECB and iv is None:
            raise ValueError("CBC 및 OFB 모드에는 IV가 필요합니다.")

        enc_data = bytes.fromhex(cipher_text)
        
        # ECB 모드일 경우
        if self.mode == AES.MODE_ECB:
            cipher = AES.new(key.encode(), self.mode)
            decrypted = cipher.decrypt(enc_data)
        else:  # CBC나 OFB 모드일 경우
            cipher = AES.new(key.encode(), self.mode, iv.encode())
            decrypted = cipher.decrypt(enc_data)

        # 결과 반환: 복호화된 plain text를 return
        return self.unpad(decrypted).decode()
        

In [3]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

In [4]:
# 2) RSA Crypto 클래스 구현 
class RSACrypto:
    def __init__(self):
        self.key_size = 1024  # 키 사이즈는 1024로 고정
        
    def Create_KeyPair(self):
        # RSA 키 쌍 생성
        key = RSA.generate(self.key_size)
        # 공개키와 개인키 추출
        public_key = key.publickey().exportKey()
        private_key = key.exportKey()
        return public_key, private_key
    
    def Encrypt(self, data, key):
        # 키 가져오기
        rsa_key = RSA.importKey(key)

        # 공개키가 아니면 에러 발생
        if not rsa_key.has_private():
            # 암호화 도구 초기화
            cipher = PKCS1_OAEP.new(rsa_key)
            # 데이터 암호화
            encrypted_data = cipher.encrypt(data.encode())
            # 암호화된 cipher_text를 return
            return encrypted_data
        else:
            raise ValueError("Private key provided for encryption, expected public key.")
    
    def Decrypt(self, data, key):
        # 키 가져오기
        rsa_key = RSA.importKey(key)

        # 개인키가 아니면 에러 발생
        if rsa_key.has_private():
            # 암호화 도구 초기화
            cipher = PKCS1_OAEP.new(rsa_key)
            # 데이터 복호화
            decrypted_data = cipher.decrypt(data).decode()
            # 복호화된 plain_text를 return
            return decrypted_data
        else:
            raise ValueError("Public key provided for decryption, expected private key.")

---
# ✅**Alice & Bob 상황 구현**

In [5]:
# 3) AES와 RSA 응용 

## 1. RSA Key Pair 생성
rsa_Alice = RSACrypto()
rsa_Bob = RSACrypto()

alice_public_key, alice_private_key = rsa_Alice.Create_KeyPair()
bob_public_key, bob_private_key = rsa_Bob.Create_KeyPair()

## 2. Alice는 AES 키를 생성
aes_Alice = AESCipher()
aes_Alice.Set_Mode(AESCipher.MODE_CBC)
aes_key = aes_Alice.Create_Key() # 기본 키 사이즈 16을 사용
iv = aes_Alice.Create_IV()

## 3. Alice는 Bob의 공개키를 사용하여 AES 키를 암호화
encrypted_aes_key = rsa_Alice.Encrypt(aes_key, bob_public_key)

## 4. Bob은 자신의 개인키로 AES 키를 복호화하여 얻음
decrypted_aes_key = rsa_Bob.Decrypt(encrypted_aes_key, bob_private_key)

assert aes_key == decrypted_aes_key, "AES Key decryption failed!"

## 5. Alice는 생성한 AES 키로 개인정보를 암호화하여 Bob에게 전송
alice_info = "Alice's Personal Info"
encrypted_info_from_alice = aes_Alice.Encrypt(alice_info, aes_key, iv)

## 6. Bob은 수신한 정보를 암호화된 AES 키로 복호화하여 Alice의 개인정보를 확인
aes_Bob = AESCipher()
aes_Bob.Set_Mode(AESCipher.MODE_CBC)
decrypted_info_from_alice = aes_Bob.Decrypt(encrypted_info_from_alice, decrypted_aes_key, iv)

## 7. 반대로, Bob도 자신의 정보를 암호화하여 Alice에게 전송
bob_info = "Bob's Personal Info"
encrypted_info_from_bob = aes_Bob.Encrypt(bob_info, decrypted_aes_key, iv)

## 8. Alice는 수신한 정보를 암호화된 AES 키로 복호화하여 Bob의 개인정보를 확인
decrypted_info_from_bob = aes_Alice.Decrypt(encrypted_info_from_bob, aes_key, iv)

## 결과 출력
print("\n[The Result]\n")
print("1. Alice's Personal Info")
print("Original Info from Alice:", alice_info)
print("Decrypted Info from Alice:", decrypted_info_from_alice)

print("\n")
print("2. Bob's Personal Info")
print("Original Info from Bob:", bob_info)
print("Decrypted Info from Bob:", decrypted_info_from_bob)

assert alice_info == decrypted_info_from_alice, "Error : Information decryption from Alice failed!"
assert bob_info == decrypted_info_from_bob, "Error : IInformation decryption from Bob failed!"



[The Result]

1. Alice's Personal Info
Original Info from Alice: Alice's Personal Info
Decrypted Info from Alice: Alice's Personal Info


2. Bob's Personal Info
Original Info from Bob: Bob's Personal Info
Decrypted Info from Bob: Bob's Personal Info
